## Excitability of Network

Goal is to construct the network such that stimuli it recieves in t_1 
has an imact on the processing of t_2, the memory of the system.

Solve for connection_probability, processing_time, firing_steps, firing_threshold, ...

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import spikey

In [ ]:
def quincience_time(w_matrix, neuron, **config):
    neuron.reset()

    for i in range(config['processing_time']):
        i_spikes = [0 if i else config['input_magnitude']]
        b_spikes = neuron >= config['firing_threshold']

        spikes = np.append(i_spikes, b_spikes)

        neuron.update()

        neuron += np.sum(w_matrix * spikes.reshape((-1, 1)), axis=0)

        if i and not np.sum(b_spikes):
            return i
        
    return i + 15

In [ ]:
## Single Configuration
NEURON = spikey.neuron.Neuron
neuron_conf = {
    'magnitude': 1,
    'processing_time': 20,
    'input_magnitude': 4,
    'firing_threshold': 2,
    'n_neurons': 200,
    'neuron_pct_inhibitory': 0,
    'potential_decay': .05,
    'refractory_period': 0,
    'resting_mv': 0,
    'spike_delay': 0,
    'prob_rand_fire': 0,
}

WEIGHT = w.RandomUni
weight_conf = {
    'connection_probability': .30,
    'max_weight': 2,
    'n_inputs': 1,
}
config = {**neuron_conf, **weight_conf}

times = []
for _ in range(1000):
    weight = WEIGHT(**config)
    neuron = NEURON(**config)

    t = quincience_time(weight, neuron, **config)

    times.append(t)

sns.distplot(times)
plt.title("Quincience Times per Static Configuration")
plt.xlabel("Time till Quiet")
plt.ylabel("%")
plt.show()

In [ ]:
NEURON = spikey.neuron.Neuron
neuron_conf = {
    'magnitude': 1,
    'processing_time': 20,
    'input_magnitude': 4,
    'firing_threshold': 2,
    'n_neurons': 100,
    'neuron_pct_inhibitory': 0,
    'potential_decay': .05,
    'refractory_period': 0,
    'resting_mv': 0,
    'spike_delay': 0,
    'prob_rand_fire': 0,
}

WEIGHT = w.RandomUni
weight_conf = {
    'connection_probability': .30,
    'max_weight': 2,
    'n_inputs': 1,
}
config = {**neuron_conf, **weight_conf}

PARAMETER = 'connection_probability'
PARAMETER_VALUES = np.linspace(0, 1, 5)
average_times = []

for param in PARAMETER_VALUES:
    config.update({PARAMETER: param})

    times = []
    for _ in range(1000):
        weight = WEIGHT(**config)
        neuron = NEURON(**config)

        t = quincience_time(weight, neuron, **config)

        times.append(t)

    average_times.append(np.mean(times))

plt.plot(PARAMETER_VALUES, average_times)
plt.title(f"Average Quincience Time vs {PARAMETER}")
plt.xlabel(f"{PARAMETER}")
plt.ylabel("Average Quincience Tim{e")
plt.show()